In [1]:
import numpy as np
import pandas as pd
from pandas_ods_reader import read_ods

df = read_ods('data/inflections_urldecoded.ods')
df.replace(to_replace=[None], value='', inplace=True)
print('done')

done


In [2]:
def get_inflection(word, basePos, root,baseGender,tense,baseNumber,basePerson,binyan,PGN, type, baseLexiconPointer,value):
    if baseGender == 'feminine':
        baseGender = 'masculine'
    elif baseGender == 'masculine':
        baseGender = 'feminine'

    if basePos == 'numeral':
        change_gender_word = df.loc[(df['basePos'] == basePos) &
           (df['root'] == root) &
           (df['baseGender'] == baseGender) &
           (df['tense'] == tense) &
           (df['basePerson'] == basePerson) &
           (df['binyan'] == binyan) &
           (df['type'] == type) &
           (df['PGN'] == PGN) &
           (df['value'] == value)
        ]
    else:
        change_gender_word = df.loc[(df['basePos'] == basePos) &
           (df['root'] == root) &
           (df['baseGender'] == baseGender) &
           (df['tense'] == tense) &
           (df['baseNumber'] == baseNumber) &
           (df['basePerson'] == basePerson) &
           (df['binyan'] == binyan) &
           (df['type'] == type) &
           (df['PGN'] == PGN) &
           (df['baseLexiconPointer'] == baseLexiconPointer)
        ]
    if len(change_gender_word) == 0:
        return None
    return change_gender_word['surface'].values[0]
    # another_inflection = df.loc[df['surface'] == word]
    # print(another_inflection)



gender_dict = {
    'feminine': 'feminine',
    'masculine': 'masculine',
    'F': 'feminine',
    'M': 'masculine'
}
tense_dict = {
    'FUTURE': 'future',
    'IMPERATIVE':'imperative',
    '-1': 'unspecified',
    '': '',
    'beinoni': 'beinoni',
    'BEINONI': 'beinoni',
    'PAST': 'past'
}
def get_inflection_specific_word(word, type, gender, tense):
    gender = gender_dict[gender]
    tense = tense_dict[str(tense)]
    if tense == '':
                another_inflection = df.loc[(df['surface'] == word) &
                                    ((df['type'] == type) | (df['basePos'] == type))
                                    & (df['baseGender'] == gender)
        ]
    else:
        another_inflection = df.loc[(df['surface'] == word) &
                                    ((df['type'] == type) | (df['basePos'] == type))
                                    & (df['baseGender'] == gender)
                                    & (df['tense'] == tense)
        ]
    if len(another_inflection) == 0:
        another_inflection = df.loc[(df['surface'] == word) &
                                    ((df['type'] == type) | (df['basePos'] == type))
                                    & (df['baseGender'] == gender)
                                    & (df['tense'] == 'beinoni')
        ]
        if len(another_inflection) == 0:
            return None
    # select_word = another_inflection.iloc[0]
    for index, select_word in another_inflection.iterrows():
        basePos = select_word['basePos']
        root = select_word['root']
        baseGender = select_word['baseGender']
        tense = select_word['tense']
        baseNumber = select_word['baseNumber']
        basePerson = select_word['basePerson']
        binyan = select_word['binyan']
        PGN = select_word['PGN']
        type = select_word['type']
        baseLexiconPointer = select_word['baseLexiconPointer']
        value = select_word['value']
        change_word = get_inflection(select_word, basePos, root, baseGender, tense, baseNumber, basePerson, binyan, PGN, type,baseLexiconPointer,value)
        if change_word != None:
            break
    else:
        change_word = None

    return change_word


In [3]:
import requests
from time import sleep

yap_token = "ca1e42cb5ee83d4e0224df9e335bcd9e"
url = f'https://www.langndata.com/api/heb_parser?token={yap_token}'

#gen,num_s_p,per,tense
def get_word_info(json_obj):
    md_lattice = json_obj["md_lattice"]
    # ma_lattice = json_obj["ma_lattice"]
    res_df = pd.json_normalize([md_lattice[i] for i in md_lattice.keys()])
    # res_df_ma_lattice = pd.json_normalize([ma_lattice[i] for i in ma_lattice.keys()])

    res_df = res_df[['word', 'lemma', 'gen', 'num_s_p', 'per', 'tense','pos']]
    # res_df_ma_lattice = res_df_ma_lattice[['word', 'lemma', 'gen', 'num_s_p', 'per', 'tense']]

    return res_df

#gen,num_s_p,per,tense
def get_word_info_hebrew_nlp(text):
    request = {
        'token': 'TW5M2ZoDwIahTGJ',
        'readable': False,
        'paragraph': text
    }
    result = requests.post('https://hebrew-nlp.co.il/service/Morphology/Analyze', json=request).json()[0]
    result = [item[0] for item in result]
    res_df = pd.json_normalize([result[i] for i in range(len(result))])
    res_df = res_df[['baseWordMenukad', 'gender', 'plural', 'person', 'tense','partOfSpeech']]
    # res_df['baseWordMenukad'] = res_df['baseWordMenukad'][:-1] + res_df['baseWordMenukad'][-1].upper()
    return res_df


In [4]:

#subject relate to the word
def is_relate_to_subject(dep_tree_df, word_ind, subject_ind):
    subject_details = dep_tree_df.iloc[subject_ind]
    return subject_details['dependency_arc'] == int(word_ind)

# word relate to PRP - no need?
# def is_relate_to_subject_PRP(dep_tree_df, word_ind, subject_ind,df_word_info):
#     word_detail = dep_tree_df.iloc[word_ind]
#     relate_word_dep_tree = dep_tree_df.loc[word_detail['dependency_arc'] == dep_tree_df['num']].iloc[0]
#     related_word_ind = relate_word_dep_tree['num']
#     relate_word_info = df_word_info.iloc[int(related_word_ind) - 1]
#     if relate_word_dep_tree['pos'] == 'PRP' and relate_word_info['per'] == 2:
#         return True
#     else:
#         return False

#PRP relate to word
def is_relate_to_subject_PRP(dep_tree_df, word_ind, subject_ind,df_word_info):
    word_detail = dep_tree_df.iloc[word_ind]
    #get all PRP with per 2
    all_PRP = df_word_info.loc[((df_word_info['pos'] == 'PRP') & (df_word_info['per'] == '2'))]
    word_num = word_ind + 1
    for index in all_PRP.index.values.tolist():
        if int(dep_tree_df.iloc[index]['dependency_arc']) == word_num:
            return True
    return False

#word relate to AT
# def is_relate_to_subject_AT(dep_tree_df, word_ind, subject_ind,df_word_info):
#     word_detail = dep_tree_df.iloc[word_ind]
#     #get all PRP with per 2
#     all_AT = df_word_info.loc[df_word_info['pos'] == 'AT']
#     word_num = word_ind + 1
#     for index in all_AT.index.values.tolist():
#         if int(dep_tree_df.iloc[word_ind]['dependency_arc']) == dep_tree_df.iloc[index]['num']:
#             return True
#     return False

#PRP relate to word
def is_relate_to_subject_PRP_not_of_per_2(dep_tree_df, word_ind, subject_ind,df_word_info):
    word_detail = dep_tree_df.iloc[word_ind]
    #get all PRP with per 2
    all_PRP = df_word_info.loc[(df_word_info['pos'] == 'PRP') & (df_word_info['per'] != '2')]
    word_num = word_ind + 1
    for index in all_PRP.index.values.tolist():
        if int(dep_tree_df.iloc[index]['dependency_arc']) == word_num:
            return True
    return False

#word relate to root and dont have any PRP from per 1/3 relate to it
def is_relate_to_subject_2(dep_tree_df, word_ind,df_word_info):
    word_detail = dep_tree_df.iloc[word_ind]
    subject_num = dep_tree_df.loc[dep_tree_df['dependency_part'] == 'ROOT'].iloc[0]['num']

    all_PRP = df_word_info.loc[(df_word_info['pos'] == 'PRP') & (df_word_info['per'] != '2')]
    word_num = word_ind + 1
    for index in all_PRP.index.values.tolist():
        if int(dep_tree_df.iloc[index]['dependency_arc']) == word_num:
            return False

    return word_detail['dependency_arc'] == subject_num


def is_relate_to_subject_bot_nn(dep_tree_df, word, wordDetail,word_ind,df_word_info):
    subject_ind = dep_tree_df.index[dep_tree_df['dependency_part'] == 'ROOT'].tolist()[0]
    dependency_arc = int(dep_tree_df.iloc[word_ind]['dependency_arc'])
    head_of_nn_dep_arc_info = df_word_info.iloc[dependency_arc - 1]
    if head_of_nn_dep_arc_info['pos'] == 'VB':
        return False
    if is_relate_to_subject_2(dep_tree_df, word_ind, df_word_info) or \
            is_relate_to_subject(dep_tree_df, word_ind, subject_ind)  or is_relate_to_subject_PRP(dep_tree_df, word_ind, subject_ind,df_word_info) or (subject_ind == word_ind and not is_relate_to_subject_PRP_of_per_1(dep_tree_df, word_ind, subject_ind,df_word_info)):
        return True
    else:
        return False


def is_relate_to_subject_bot_jj(dep_tree_df, word, wordDetail,word_ind,df_word_info):
    subject_ind = dep_tree_df.index[dep_tree_df['dependency_part'] == 'ROOT'].tolist()[0]
    if is_relate_to_subject_2(dep_tree_df, word_ind, df_word_info) or \
            is_relate_to_subject(dep_tree_df, word_ind, subject_ind)  or is_relate_to_subject_PRP(dep_tree_df, word_ind, subject_ind,df_word_info) or (subject_ind == word_ind and not is_relate_to_subject_PRP_not_of_per_2(dep_tree_df, word_ind, subject_ind,df_word_info)):
        return True
    else:
        return False

def is_relate_to_subject_bot(dep_tree_df, word, wordDetail,wodIndex,df_word_info):
    #wordDetail['per'] == 'A or
    if wordDetail['per'] == '2':
        return True
    if wordDetail['per'] == 'A':
        return is_relate_to_subject_bot_jj(dep_tree_df, word, wordDetail,wodIndex,df_word_info)

        # if is_relate_to_subject_2(dep_tree_df, word, subject) or is_relate_to_subject(dep_tree_df, word, subject) or subject == wodIndex:
        #     return True
        # else:
        #     return False
    return False
    # return word_detail['dependency_arc'] == subject_num



In [5]:
def get_req_from_yap(text):
    text = text.replace(r'"', r'\"')
    _json = '{"data":"' + text.strip() + '"}'
    sleep(3)
    r = requests.post(url, data=_json.encode('utf-8'), headers={'Content-type': 'application/json; charset=utf-8'})
    json_obj = r.json()
    return json_obj

def get_dep_tree(json_obj):
    dep_tree = json_obj["dep_tree"]
    res_df = pd.json_normalize([dep_tree[i] for i in dep_tree.keys()])
    return res_df


personal_pronoun_dict = {
    'אתה': 'את',
    'את':'אתה',
    'אתם':'אתן',
    'אתן':'אתם',
    'כן' : 'כם',
    'כם' : 'כן',
    'תן' : 'תם',
    'תם': 'תן',
    # 'הם':'הן',
    # 'הן':'הם',
    # 'הוא':'היא',
    # 'היא':'הוא',
}

def is_PRP(word_info):
    pos = word_info['pos']
    if 'PRP' in pos or 'PRN' in pos:
        return True
    # elif pos == 'AT':
    #     return True
    else:
        return False


def get_untokenize(text, textArr, text_arr_is_change, df_word_info):
    text_split_native = text.split(' ')
    good_textArr = []
    current_word = ""
    for i, word in enumerate(textArr):
        word_info = df_word_info.iloc[i]
        if current_word != '' and word_info['pos'] == 'DEF':
            continue
        current_word += word
        if word_info['pos'] in ['COM','TO','WDT','WP','PREPOSITION','DEF','REL','CONJ','TEMP'] \
                and not text_arr_is_change[i] \
                and (word_info['gen'] == -1 or word_info['num_s_p'] == -1 or word_info['per'] == -1 or word_info['tense'] == -1)\
                and len(word) < 3: #and word not in text_split_native
            continue
        else:
            good_textArr.append(current_word)
            current_word = ""
    good_textArr[-1] += current_word
    return good_textArr


In [6]:
def change_sentence_gender(text,show_print=False):
    text_by_sentence = text.split('\n\n')
    json_obj = get_req_from_yap(text_by_sentence[0])
    df_word_info = get_word_info(json_obj)
    dep_tree = get_dep_tree(json_obj)
    word_info_2 = get_word_info_hebrew_nlp(text)
    textArr = df_word_info['word'].tolist()
    text_arr_is_change = [False] * len(textArr)
    for i, token in enumerate(df_word_info['word']):
        # ret = is_relate_to_subject_2(dep_tree,token,'נעם')
        # ret = is_relate_to_subject(dep_tree,token,'אליך')
        # try:
        # word_info = df_word_info.loc[df_word_info['word'] == token].iloc[0]
        word_info = df_word_info.iloc[i]
        is_jj = (word_info['pos'] == 'JJ')
        is_verb = (word_info['pos'] in ['VB','BD','BG','BN','BP','BZ',''])
        is_modal = (word_info['pos'] == 'MD')
        is_num = (word_info['pos'] == 'CD')
        is_noun = (word_info['pos'] == 'NN')
        word_gender = word_info['gen']
        # except:
        #     continue
        if is_noun:
            ret = is_relate_to_subject_bot_nn(dep_tree, token, word_info,i,df_word_info)
        elif is_jj or is_num:
            ret = is_relate_to_subject_bot_jj(dep_tree, token, word_info,i,df_word_info)
        elif is_verb or is_modal:
            # if word_info['pos'] == 'BN':
            #     ret = is_relate_to_subject_bot_jj(dep_tree, token, word_info,i,df_word_info)
            # else:
            ret = is_relate_to_subject_bot(dep_tree, token, word_info,i,df_word_info)
        elif is_PRP(word_info):
            ret = False
            if token in personal_pronoun_dict:
                new_word = personal_pronoun_dict[token]
            else:
                new_word = token
            # dep_tree[dep_tree['word'] == token] = new_word
            textArr[i] = new_word
            text_arr_is_change[i] = True
        else:
            ret = False
        if show_print:
            print(token, ' : ', ret)
        if ret:
            # get_inflection_specific_word('הולכת',type='verb', gender='F', tense='beinoni')
            if is_jj:
                new_word = get_inflection_specific_word(token, type='adjective', gender=word_gender, tense='')
            elif is_noun:
                new_word = get_inflection_specific_word(token, type='noun', gender=word_gender, tense='')
            elif is_num:
                new_word = get_inflection_specific_word(token, type='numeral', gender=word_gender, tense='')
            elif is_verb:
                if word_info['pos'] == 'BN':
                    new_word = get_inflection_specific_word(token, type='verb', gender=word_gender,
                    tense='beinoni'
                    )
                new_word = get_inflection_specific_word(token, type='verb', gender=word_gender,
                tense=word_info['tense']
                )
            elif is_modal:
                new_word = get_inflection_specific_word(token, type='modal', gender=word_gender,
                tense = word_info['tense']
                # tense='unspecified'
                )

            # dep_tree[dep_tree['word'] == token] = new_word
            textArr[i] = new_word
            text_arr_is_change[i] = True


    textArr = [word if word != None else "None" for word in textArr]

    good_textArr = get_untokenize(text, textArr, text_arr_is_change, df_word_info)


    new_sentence = " ".join(good_textArr)
    # print(dep_tree['word'])
    return new_sentence



In [8]:
text = 'אתם חמישה עובדים'
print(f'origin text: {text}')
print(f'text after gender rewriting: {change_sentence_gender(text)}')
print('\n\n')

origin text: אתם חמישה עובדים
text after gender rewriting: אתן חמש עובדות



